In [95]:
import glob
import json
import os
import subprocess

import geopandas
import pandas
from tqdm.notebook import tqdm

In [9]:
with open("../config.json") as fh:
    config = json.load(fh)
config

{'base_path': '/home/tom/OneDrive/ghana-oxford-infrastructure-adaptation-2020/'}

In [81]:
flood_risk_rasters = glob.glob(os.path.join(
    config['base_path'], 'incoming', 'hazards', 'Maps_Eric', 'flood_hazard', 'ID*'))

flood_risk_rasters

['/home/tom/OneDrive/ghana-oxford-infrastructure-adaptation-2020/incoming/hazards/Maps_Eric/flood_hazard/ID73_flood_future_eric',
 '/home/tom/OneDrive/ghana-oxford-infrastructure-adaptation-2020/incoming/hazards/Maps_Eric/flood_hazard/ID72_flood_current_eric']

In [83]:
"Id_future_asda".contains("future")

AttributeError: 'str' object has no attribute 'contains'

In [85]:
for raster_path in flood_risk_rasters:
    if "future" in os.path.basename(raster_path):
        name = "future"
    else:
        name = "current"
        
    print(name)
    tile_dir = os.path.join(
        config['base_path'], 'data', 'hazards', name, 'tiles')
    
    sp = subprocess.run(["mkdir", "-p", tile_dir], capture_output=True)
    print(sp.stdout.decode('utf-8'))
                         
    sp = subprocess.run(
        ["gdal_retile.py", "-ps", "1024", "1024", "-targetDir", tile_dir, raster_path], 
        capture_output=True)
    print(sp.stdout.decode('utf-8'))

future

0...10...20...30...40...50...60...70...80...90...100 - done.

current

0...10...20...30...40...50...60...70...80...90...100 - done.



In [86]:
for raster_path in flood_risk_rasters:    
    if "future" in os.path.basename(raster_path):
        name = "future"
    else:
        name = "current"
    
    tile_dir = os.path.join(config['base_path'], 'data', 'hazards', name, 'tiles')
    
    threshold_dir = os.path.normpath(os.path.join(tile_dir, '..', 'thresholds'))
    subprocess.run(["mkdir", "-p", threshold_dir])
                   
    vector_dir = os.path.normpath(os.path.join(tile_dir, '..', 'polygons'))
    subprocess.run(["mkdir", "-p", vector_dir])
    
    for tile_path in glob.glob(tile_dir + '/*.tif'):
        tile_name = os.path.basename(tile_path).replace(".tif", "")
        
        # thresholds = [0.0, 0.25, 0.5, 1.0, 2.0, 3.0, 5.0, 10.0, 999.0]
        thresholds = [3,5,7]
        
        for min_depth, max_depth in zip(thresholds, thresholds[1:]):    
            # Calculate mask where raster values are between thresholds
            sp = subprocess.run([
                "gdal_calc.py",
                "-A", 
                f"{tile_dir}/{tile_name}.tif",
                f"--outfile={threshold_dir}/{tile_name}_{min_depth}m{max_depth}m.tif",
                f"--calc=logical_and(A>={min_depth},A<{max_depth})",
                "--format=GTiff",
                "--type=Byte",
                "--NoDataValue=0",
                "--co=SPARSE_OK=YES",
                "--quiet",
                "--co=COMPRESS=LZW",
            ], check=True)

            # Convert raster masks to vector polygons
            sp = subprocess.run([
                "gdal_polygonize.py",
                f"{threshold_dir}/{tile_name}_{min_depth}m{max_depth}m.tif",
                "-q",
                "-f", 
                "GPKG",
                f"{vector_dir}/{tile_name}_{min_depth}m{max_depth}m.gpkg"
            ], check=True)

In [96]:
for raster_path in flood_risk_rasters:    
    if "future" in os.path.basename(raster_path):
        name = "future"
    else:
        name = "current"
    
    vector_dir = os.path.join(config['base_path'], 'data', 'hazards', name, 'polygons')
    
    for min_depth, max_depth in zip(thresholds, thresholds[1:]):    
        match = os.path.join(vector_dir, f"*{min_depth}m{max_depth}m.gpkg")
        tiles = glob.glob(match)
        
        dfs = []
        for tile in tqdm(tiles):
            df = geopandas.read_file(tile)
            dfs.append(df)
            
        df = pandas.concat(dfs)
        df.to_file(
            os.path.join(config['base_path'], 'data', 'hazards', f"{name}_{min_depth}-{max_depth - 1}.gpkg"),
            driver="GPKG"
        )
    

In [101]:
for raster_path in flood_risk_rasters:    
    if "future" in os.path.basename(raster_path):
        name = "future"
    else:
        name = "current"
    
    # Clean up - remove working directories
    working_dir = os.path.join(config['base_path'], 'data', 'hazards', name)
    subprocess.run(["rm", "-r", working_dir])